# Group all top six language books #

In [110]:
import os
import numpy as np
import json

dic = json.load(open("/usr/local/NNShared/hali/data/cleanBL.json", 'r'))

en = {}
fr = {}
fi = {}
nl = {}
de = {}
pt = {}

for books, languages in dic.items():
    if languages.lower() == 'english':
        en[books] = languages
    elif languages.lower() == 'englishs':
        en[books] = 'english'
    elif languages.lower() == 'en':
        en[books] = 'english'
    elif languages.lower() == 'french':
        fr[books] = languages
    elif languages.lower() == 'finnish':
        fi[books] = languages
    elif languages.lower() == 'dutch':
        nl[books] = languages
    elif languages.lower() == 'german':
        de[books] = languages
    elif languages.lower() == 'portuguese': 
        pt[books] = languages
    elif languages.lower() == 'portugese':
        pt[books] = 'portuguese'
        
topsix = {
    'en' : en,
    'fr' : fr,
    'fi' : fi,
    'nl' : nl,
    'de' : de,
    'pt' : pt
}

print(len(topsix))

with open("/usr/local/NNShared/hali/data/topsix.json", 'w') as f:
    json.dump(topsix, f)

6


# Define the function for getting characters from En books #

In [1]:
from random import *
import json

### load the json dictionary
dic = json.load(open("/usr/local/NNShared/hali/data/topsix.json", 'r'))

In [2]:
def Encharacter():
    texts = []
    labels = []
    
    ### grab the books and labels
    for books, languages in dic['en'].items():
        labels.append(languages)
        with open("/usr/local/NNShared/hali/newebooks/"+books+".txt","r") as f:
            maxlen = len(f.read())
            f.seek(0) ### reset f.read() ##
            
            randpt = randrange(round((maxlen-30)/3), maxlen-30) ### grab the characters, starting from 1/3 position to avoid preface ###
            char = f.read()[randpt:randpt+30]
            texts.append(char)
            f.close()
    return texts, labels

# Define the function for getting characters from non-En books #

In [3]:
def Nonencharacter(non_en_dic):
    non_en_texts = []
    non_en_labels = []
    n = int(len(dic['en'])/len(non_en_dic))
    
    ### grab the books and labels
    for books, languages in non_en_dic.items():
        with open("/usr/local/NNShared/hali/newebooks/"+books+".txt","r") as f:
            maxlen = len(f.read())
            f.seek(0)   
        
            for i in range(n): ### grab 30 character from n random sequences from the book ###
                randpt = randrange(round((maxlen-30)/3), maxlen-30) ### grab the characters, starting from 1/3 position to avoid preface ###
                char = f.read()[randpt:randpt+30]
                non_en_texts.append(char)
                non_en_labels.append(languages)
            
                f.seek(0)  
            
            f.close()
    return non_en_texts, non_en_labels

In [4]:
texts, labels = Encharacter()

In [5]:
non_en_texts1, non_en_labels1 = Nonencharacter(dic['fr'])
non_en_texts2, non_en_labels2 = Nonencharacter(dic['fi'])
non_en_texts3, non_en_labels3 = Nonencharacter(dic['nl'])
non_en_texts4, non_en_labels4 = Nonencharacter(dic['de'])
non_en_texts5, non_en_labels5 = Nonencharacter(dic['pt'])

#  Vectorize each character in each book #

In [6]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer

In [7]:
def string_vectorizer(strng, dic):
    vector = [[0 if char != letter.lower() else 1 for char in dic] 
                for letter in strng]
    return vector

In [8]:
def chartovec(characters):
    tokenizer = Tokenizer(char_level=True)
    tokenizer.fit_on_texts(characters)
    sequences = tokenizer.texts_to_sequences(characters)
    one_hot_results = tokenizer.texts_to_matrix(characters, mode='binary')
    dic = tokenizer.word_index
    
    x_input = []
    
    for i in range(len(characters)):
    
        x_input.append(np.reshape(string_vectorizer(characters[i], dic),(30,len(dic)))) 
    
    return np.array(x_input)

# Vectorize the labels #

In [9]:
def wordtovec(tags):
    tokenizer1 = Tokenizer(num_words = 1000)
    tokenizer1.fit_on_texts(tags)
    sequences1 = tokenizer1.texts_to_sequences(tags)
    one_hot_results1 = tokenizer1.texts_to_matrix(tags, mode='binary')
    dic1 = tokenizer1.word_index
    y_input = np.array(string_vectorizer(tags, dic1))
    
    return y_input

# results #

In [10]:
characters = texts + non_en_texts1 + non_en_texts2 + non_en_texts3 + non_en_texts4 + non_en_texts5
tags = labels + non_en_labels1 + non_en_labels2 + non_en_labels3 + non_en_labels4 + non_en_labels5

In [11]:
X_input = chartovec(characters)

In [15]:
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(characters)
sequences = tokenizer.texts_to_sequences(characters)
one_hot_results = tokenizer.texts_to_matrix(characters, mode='binary')
dic = tokenizer.word_index
print(dic)

{' ': 1, 'e': 2, 'a': 3, 'n': 4, 'i': 5, 't': 6, 's': 7, 'o': 8, 'r': 9, 'l': 10, 'd': 11, 'u': 12, 'h': 13, 'm': 14, 'c': 15, ',': 16, 'p': 17, 'g': 18, 'k': 19, 'v': 20, '.': 21, 'b': 22, 'f': 23, 'ä': 24, 'w': 25, 'j': 26, 'y': 27, 'z': 28, '-': 29, 'q': 30, "'": 31, 'é': 32, '"': 33, '_': 34, 'x': 35, ';': 36, '!': 37, 'ã': 38, 'ö': 39, 'ü': 40, ':': 41, 'ç': 42, '1': 43, '?': 44, 'à': 45, 'ß': 46, 'è': 47, 'á': 48, ')': 49, '2': 50, '0': 51, '(': 52, 'ê': 53, '»': 54, '«': 55, '3': 56, '8': 57, '5': 58, '4': 59, '*': 60, ']': 61, '[': 62, '6': 63, '7': 64, 'ó': 65, '9': 66, 'õ': 67, '|': 68, 'ô': 69, '=': 70, 'â': 71, 'î': 72, 'ù': 73, 'û': 74, '\xa0': 75, '~': 76, '+': 77, 'ë': 78, 'í': 79, '&': 80, 'ï': 81, '/': 82, '©': 83, '^': 84, '{': 85, '#': 86, '}': 87, 'ú': 88, 'º': 89, '·': 90, '$': 91, 'ª': 92, 'æ': 93, '\x92': 94, '§': 95, '°': 96, '>': 97, '<': 98, 'ò': 99, '¨': 100, 'å': 101, '\t': 102, '´': 103, '¤': 104, 'ñ': 105, '%': 106, '¡': 107, 'ì': 108, '`': 109, 'þ': 110, 

In [16]:
with open("/usr/local/NNShared/hali/data/characterdic.json", 'w') as f:
    json.dump(dic, f)

In [17]:
Y_input = wordtovec(tags)

In [18]:
tokenizer1 = Tokenizer(num_words = 1000)
tokenizer1.fit_on_texts(tags)
sequences1 = tokenizer1.texts_to_sequences(tags)
one_hot_results1 = tokenizer1.texts_to_matrix(tags, mode='binary')
dic1 = tokenizer1.word_index

with open("/usr/local/NNShared/hali/data/tagdic.json", 'w') as f:
    json.dump(dic1, f)

In [19]:
from sklearn.utils import shuffle
X_input, Y_input = shuffle(X_input, Y_input)

In [20]:
#save the data 
with open("/usr/local/NNShared/hali/data/input_X.npy", "wb") as f:
    np.save(f, X_input)

In [21]:
#save the data 
with open("/usr/local/NNShared/hali/data/input_Y.npy", "wb") as f:
    np.save(f, Y_input)